# RFA Proyect
Training MLP models over Diabetic_Retinopathy dataset step 4
- Analysis over a CNN

## 4.0. Preprocess

In [1]:
! pip install keras_tuner
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import keras
import keras_tuner

keras.utils.set_random_seed(23)
import tensorflow as tf
import numpy as np

!export TF_CUDNN_WORKSPACE_LIMIT_IN_MB=1024
from datasets import load_dataset, DatasetDict, ClassLabel

In [12]:
def prepare_inputs(image_label):
    image = tf.cast(image_label["image"], tf.float32)
    image = tf.image.resize(image, [128, 128])
    return (image, tf.keras.utils.to_categorical(image_label["label"], num_classes=5))


ds = load_dataset("Rami/Diabetic_Retinopathy_Preprocessed_Dataset_256x256").shuffle(
    seed=23
)
ds = ds.cast_column(
    "label",
    ClassLabel(
        names=["Healthy", "Moderate DR", "Mild DR", "Proliferate DR", "Severe DR"]
    ),
)
ds = (
    ds["train"].to_tf_dataset().map(prepare_inputs, num_parallel_calls=tf.data.AUTOTUNE)
)
trainval, test = keras.utils.split_dataset(ds, right_size=0.20)
train, val = keras.utils.split_dataset(trainval, right_size=0.20)

In [13]:
ds_train = train.cache()
ds_train = ds_train.shuffle(buffer_size=1000)
ds_train = ds_train.batch(128)
train_dataset = ds_train.prefetch(tf.data.AUTOTUNE)
ds_val = val.batch(128)
ds_val = ds_val.cache()
val_dataset = ds_val.prefetch(tf.data.AUTOTUNE)
ds_test = test.batch(128)
ds_test = ds_test.cache()
test_dataset = ds_test.prefetch(tf.data.AUTOTUNE)

In [14]:
for e in train_dataset.take(1):
    print(f"After preprocesss Image type {type(e[0])}")
    print(f"After preprocesss Input shape: {e[0].shape}")
    input_dim = tuple(e[0].shape.as_list()[1:])
    num_classes = 5

After preprocesss Image type <class 'tensorflow.python.framework.ops.EagerTensor'>
After preprocesss Input shape: (128, 128, 128, 3)


## 4.1. Simple CNN

Resultados:
- Test: Accuracy 72.00% con lr=0.00146
- Validation: Accuracy 69% con lr=0.00146

60% con UnINT data

66.55% con Float32 data


In [15]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        inputs = keras.Input(shape=input_dim)
        inputs = keras.layers.Rescaling(1.0 / 255)(inputs)
        filters = 32
        conv = keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu")(
            inputs
        )
        pooling = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
        conv = keras.layers.Conv2D(2 * filters, kernel_size=(3, 3), activation="relu")(
            pooling
        )
        pooling = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
        dropout = 0.5
        x = keras.layers.Flatten()(pooling)
        x = keras.layers.Dense(units=500, activation="relu")(x)
        x = keras.layers.Dropout(dropout)(x)
        predictions = keras.layers.Dense(num_classes, activation="softmax")(x)
        M = keras.models.Model(inputs=inputs, outputs=predictions)
        learning_rate = hp.Float("learning rate", min_value=0.001, max_value=0.002)
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
        M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
        return M

    def fit(self, hp, M, train_dataset, val_dataset, **kwargs):
        factor = 0.38
        patience = 5
        reduce_cb = keras.callbacks.ReduceLROnPlateau(
            monitor="val_accuracy",
            factor=factor,
            patience=patience,
            min_delta=0.0,
            min_lr=0.0,
        )
        early_cb = keras.callbacks.EarlyStopping(
            monitor="val_accuracy", patience=2 * patience, min_delta=0
        )
        kwargs["callbacks"].extend([reduce_cb, early_cb])
        return M.fit(
            train_dataset,
            batch_size=128,
            epochs=100,
            validation_data=val_dataset,
            **kwargs
        )

In [16]:
tuner = keras_tuner.BayesianOptimization(
    MyHyperModel(),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="/tmp",
    project_name="CIFAR-10",
)

In [17]:
tuner.search(train_dataset, val_dataset)

Trial 10 Complete [00h 01m 23s]
val_accuracy: 0.6636363863945007

Best val_accuracy So Far: 0.6795454621315002
Total elapsed time: 00h 16m 44s


In [18]:
tuner.results_summary(num_trials=3)

Results summary
Results in /tmp/CIFAR-10
Showing 3 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
learning rate: 0.001402995743906022
Score: 0.6795454621315002

Trial 07 summary
Hyperparameters:
learning rate: 0.0010579804845560251
Score: 0.675000011920929

Trial 02 summary
Hyperparameters:
learning rate: 0.0010669683136382809
Score: 0.6727272868156433


In [19]:
num_models = 10
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
    values = best_hyperparameters[m].values
    score = best_models[m].evaluate(test_dataset, verbose=0)
    print(
        f"Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}"
    )

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model 0: Hyperparameters: {'learning rate': 0.001402995743906022} Loss: 1.133 Precisión: 65.45%
Model 1: Hyperparameters: {'learning rate': 0.0010579804845560251} Loss: 1.191 Precisión: 63.09%
Model 2: Hyperparameters: {'learning rate': 0.0010669683136382809} Loss: 1.466 Precisión: 62.73%
Model 3: Hyperparameters: {'learning rate': 0.0010732475286180136} Loss: 1.165 Precisión: 63.45%
Model 4: Hyperparameters: {'learning rate': 0.0015776281875170385} Loss: 1.787 Precisión: 64.55%
Model 5: Hyperparameters: {'learning rate': 0.001406768538319277} Loss: 0.993 Precisión: 62.91%
Model 6: Hyperparameters: {'learning rate': 0.0013704830065525296} Loss: 1.25 Precisión: 66.55%
Model 7: Hyperparameters: {'learning rate': 0.001395208098085437} Loss: 1.356 Precisión: 62.73%
Model 8: Hyperparameters: {'learning rate': 0.001940087336024002} Loss: 1.057 Precisión: 63.82%
Model 9: Hyperparameters: {'learning rate': 0.0014189137557352148} Loss: 1.203 Precisión: 60.91%
